In [9]:
%pip install -r requirements.txt

  Using cached yfinance-0.2.63-py2.py3-none-any.whl.metadata (5.8 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached frozendict-2.4.6-py312-none-any.whl.metadata (23 kB)
  Using cached peewee-3.18.1.tar.gz (3.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.t

# Imports

In [1]:
import os
import json

from typing import List
from dotenv import load_dotenv

from pydantic import BaseModel, Field

import requests

In [2]:
from langchain.chains import SimpleSequentialChain
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableMap

In [11]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI

In [ ]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool

In [4]:
from langfuse import Langfuse

# Load Config

In [5]:
# Load environment variables from .env file
load_dotenv()

# Access environment variables
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
langfuse_public = os.getenv("LANGFUSE_PUBLIC_KEY")
langfuse_secret = os.getenv("LANGFUSE_SECRET_KEY")

In [6]:
# Set the environment variables
os.environ['AZURE_OPENAI_API_KEY'] = azure_api_key
os.environ['AZURE_OPENAI_ENDPOINT'] = azure_endpoint

# LLM Configuration

In [7]:
# Azure OpenAI - GPT-4o or GPT-4o-mini
llm = AzureChatOpenAI(
    deployment_name=deployment_name,
    model_name="gpt-4o",
    temperature=0.3,
    api_version="2024-05-01-preview",
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


# Output Schema

In [ ]:
# Define the output schema using Pydantic
class SentimentOutput(BaseModel):
    company_name: str
    stock_code: str
    newsdesc: str
    sentiment: str
    people_names: List[str]
    places_names: List[str]
    other_companies_referred: List[str]
    related_industries: List[str]
    market_implications: str
    confidence_score: float = Field(ge=0.0, le=1.0)

# Parser
parser = PydanticOutputParser(pydantic_object=SentimentOutput)

# Initialize Langfuse

In [ ]:
# Initialize Langfuse
langfuse = Langfuse(public_key=langfuse_public, secret_key=langfuse_secret)

# Get Relevant Data

In [ ]:
# Fetch symbol from company name
def get_stock_ticker(company_name):
    url = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=url, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

In [ ]:
# Fetch Yahoo Finance news
def fetch_news(ticker: str) -> str:
    tool = YahooFinanceNewsTool()
    return tool.run(ticker)

# Setup Chain

In [ ]:
# Step 4: Sentiment analysis prompt
sentiment_prompt = PromptTemplate(
    input_variables=["company", "ticker", "news"],
    template="""
Analyze the sentiment of the news for the given company.

Company: {company}
Ticker: {ticker}
News Summaries:
{news}

Return the result using this JSON format:
{format_instructions}
""",
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
# Final sentiment chain
sentiment_chain = sentiment_prompt | llm | parser

# Main Pipeline Function

In [ ]:
def run_sentiment_pipeline(company_name: str):
    trace = langfuse.trace(name="sentiment_analysis_pipeline", input={"company": company_name})

    # Extract ticker
    ticker = get_stock_ticker(company_name)
    trace.span(name="stock_code_extraction").log_input_output(input=company_name, output=ticker)

    # Fetch news
    news = fetch_news(ticker)
    trace.span(name="news_fetching").log_input_output(input=ticker, output=news)

    # Run sentiment analysis
    input_data = {
        "company": company_name,
        "ticker": ticker,
        "news": news
    }
    result = sentiment_chain.invoke(input_data)
    trace.span(name="sentiment_parsing").log_input_output(input=input_data, output=result)

    return result

# Test Case

In [ ]:
company = input("Enter company name: ")
result = run_sentiment_pipeline(company)
print(json.dumps(result.dict(), indent=2))